In [ ]:
from google.colab import files
uploaded = files.upload()


Saving adult_mod.csv to adult_mod.csv


In [39]:
import csv
import random
import math

# loading datasets and removing missing values
data = []
with open("adult_mod.csv", "r") as file:
    csv_reader = csv.reader(file)
    headers = next(csv_reader)
    for row in csv_reader:
        if '?' not in row:
            data.append(row)

sample_size = int(0.1 * len(data))

# shuffling
random_seed = random.randint(1, 1000)
random.seed(random_seed)
data_sampled = random.sample(data, sample_size)

#categorical columns (non-numeric)
categorical_columns = [1, 2, 4, 5, 6, 7, 8, 12, 13]

#One-hot encode categorical data
def one_hot_encode(data):
    encoded_data = []
    for row in data:
        encoded_row = []
        for i, value in enumerate(row[:-1]):
            if i in categorical_columns:
                unique_values = set(data_sampled[j][i] for j in range(len(data_sampled)))
                for unique_value in unique_values:
                    encoded_row.append(1 if unique_value == value else 0)
            else:
                encoded_row.append(float(value))
        encoded_row.append(row[-1])  # append label
        encoded_data.append(encoded_row)
    return encoded_data

data_encoded = one_hot_encode(data_sampled)

#Euclidean distance
def euclidean_distance(point1, point2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))

# split 80-20 test train
split_ratio = 0.8
split_index = int(split_ratio * len(data_encoded))
train_data, test_data = data_encoded[:split_index], data_encoded[split_index:]

# KNN classifier
def knn_classifier(train_data, test_point, k):
    distances = []
    for train_point in train_data:
        distance = euclidean_distance(train_point[:-1], test_point[:-1])
        distances.append((train_point, distance))
    distances.sort(key=lambda x: x[1])
    neighbors = [point[0] for point in distances[:k]]
    neighbor_genders = [point[-1] for point in neighbors]
    return max(set(neighbor_genders), key=neighbor_genders.count)

# calculating for k = 3 and k = 5
k_values = [3, 5]
for k in k_values:
    correct_predictions = 0
    for test_point in test_data:
        predicted_gender = knn_classifier(train_data, test_point, k)
        if predicted_gender == test_point[-1]:
            correct_predictions += 1
    accuracy = correct_predictions / len(test_data)
    print(f"Accuracy with K={k}: {accuracy:.2f}")

Accuracy with K=3: 0.83
Accuracy with K=5: 0.84
